<a href="https://colab.research.google.com/github/frankbass3/RareClassesPrediction/blob/main/STANFORD_Rarecalsses_IMAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%ls /content/drive/MyDrive/Colab\ Notebooks/DATI-stanford-test-immagini/archive/

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/DATI-stanford-test-immagini/archive/

In [ ]:
# df = pd.read_csv("data_example.csv")
from PIL import Image
im = Image.open("./buffalo/002.jpg")
im.show()
im

In [ ]:
%rm ./buffalo/*.txt
%rm ./elephant/*.txt
%rm ./rhino/*.txt
%rm ./zebra/*.txt
%rm ./outer-model.png
%rm ./model.png

In [ ]:
%ls ./


In [ ]:
import tensorflow as tf
batch_size = 32
img_height = 224
img_width = 224

train_list_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './',
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './',
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:

class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
      
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(repetitions):
          
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,
                                                               self.kernel_size,
                                                               activation = 'relu',
                                                               padding = 'same')
      
        # Define the max pool layer that will be added after the Conv2D blocks
        # Note on pool_size and strides:
        # If only one integer is specified, the same window length will be used for both dimensions.
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size,
                                                  strides=strides,
                                                 padding='same')

    def call(self, inputs):
        # access the class's conv2D_0 layer
        print(inputs)
        conv2D_0 = vars(self)['conv2D_0']
      
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
          
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
      
        return max_pool

In [ ]:
from tensorflow.keras.models import Model

class NotAuxiliaryModel(tf.keras.Model):

  def __init__(self, num_classes, pseudo_lables, pretrained_model): # andrebbe aggiunto num_classes per le pseudo
    super(NotAuxiliaryModel, self).__init__()

      # Creating blocks of VGG with the following 
      # (filters, kernel_size, repetitions) configurations
    self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
    self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
    self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
    self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
    self.block_e = Block(filters=512, kernel_size=3, repetitions=3)
        
      # Classification head
      # Define a Flatten layer
    self.flatten = tf.keras.layers.Flatten()

      # Create a Dense layer with 256 units and ReLU as the activation function
    self.fc = tf.keras.layers.Dense(256, activation='relu')
    # Finally add the softmax classifier using a Dense layer
    self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax') 
    # 

    self.pseudo_lables = pseudo_lables
    self.classifier_aux = tf.keras.layers.Dense(pseudo_lables, activation='softmax') # 
    self.classifier_pseudo = pretrained_model

  def call(self, inputs):
    x = self.block_a(inputs) 
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)


    output_main = self.classifier(x)#,name='output_1')
    output_aux = self.classifier_aux(x)#,name='output_2')
    pseudo_labels = self.classifier_pseudo(inputs)


    return Model(inputs = inputs, outputs=[output_main,output_aux,pseudo_labels])  #x , x_aux

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

batch_size = 32
img_height = 224
img_width = 224

tds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

vds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)




In [ ]:
# for x,y in train_list_ds:
#   print(y)
#   break
from IPython.display import Image, display
import matplotlib.pyplot as plt
for x,y in tds:
  print(x.shape)
  for xx in x:
    print(xx.shape)
    plt.imshow(xx.numpy().astype('uint8'))
    plt.show()
    break
  break

In [ ]:
total_x = []
total = []
for x,ba_y in tds:
  list_y = []
  for y in ba_y:
    if y in [1,2,3,4]:
      list_y.append(0)
    else:
      list_y.append(1)
  
  if ba_y.shape != 32:
    continue
  total.append(list_y)
  total_x.append(x)
  # print(x.shape)
  # print(ba_y.shape)

train_list_ds = tf.data.Dataset.from_tensor_slices((total_x, total))

total_x = []
total = []
for x,ba_y in vds:
  list_y = []
  for y in ba_y:
    if y in [1,2,3,4]:
      list_y.append(0)
    else:
      list_y.append(1)
  
  if ba_y.shape != 32:
    continue

  total.append(list_y)
  total_x.append(x)

val_ds = tf.data.Dataset.from_tensor_slices(( total_x, total))


In [ ]:
print(len(list(train_list_ds.as_numpy_iterator())))

In [ ]:
#dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
import tensorflow_hub as hub


module_selection = ("mobilenet_v2_100_224", 224)
handle_base, pixels = module_selection
do_fine_tuning = False
IMAGE_SIZE = (pixels, pixels)
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/classification/4".format(handle_base)


pretrainmodel = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, output_shape=[1001], trainable=do_fine_tuning),
    # tf.keras.layers.Dropout(rate=0.2),
    # tf.keras.layers.Dense(1280,
    #                       kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
pretrainmodel.build((None,)+IMAGE_SIZE+(3,))
pretrainmodel.summary()


In [ ]:
from tensorflow.python.keras.utils.vis_utils import plot_model
plot_model(pretrainmodel, show_shapes=True, show_layer_names=True)

In [ ]:
numberofpsudoclass = 1001 

def custom_loss(lam):
  def loss(y_true, y_pred):
    # Do your loss on your subset
      y_pred_n_aux, y_pred_aux, pseudo_lables  = y_pred[:, :1], y_pred[:, 1:numberofpsudoclass+1], y_pred[:, numberofpsudoclass+1:]
      cce1 = tf.keras.losses.CategoricalCrossentropy()
      cce2 = tf.keras.losses.CategoricalCrossentropy()

      l1 = cce1(y_true, y_pred_n_aux)
      #tf.print(pseudo_lables)

      l2 = cce2(pseudo_lables, y_pred_aux)
      #l1 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_pred_n_aux, logits=y_true)
      #l2 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_pred_aux,logits=pseudo_lables)
      return (l1 + lam*l2) 
  return loss

In [ ]:
import tensorflow_addons as tfa
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K

def custom_layer(pseudo_lables):
  #tf.print(input[0])
  col_indices = tf.cast(tf.argmax(pseudo_lables, axis=1), tf.int32)
  rows = tf.range(tf.shape(tf.argmax(pseudo_lables, axis=1))[0])
  sta = tf.stack([rows,col_indices], axis=1)
  # tf.print(tf.shape(pseudo_lables))
  # tf.print(tf.shape(rows))
  # tf.print(tf.shape(col_indices))
  # tf.print(tf.shape(sta))
  pred_max = tf.compat.v1.sparse_to_dense(sta,tf.shape(pseudo_lables),default_value=0.0,sparse_values=1.0)
  return pred_max

def custom_layer2(input):
  tf.print(input)
  return input

METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Accuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tfa.metrics.F1Score(name='f1', num_classes=2)
]



num_classes = 1
pseudo_lab_class = 1001

input = Input(shape=(224, 224,3), name="base_input")
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
# x = Block(filters=64, kernel_size=3, repetitions=2)(x)
# x = Block(filters=128, kernel_size=3, repetitions=2)(x)
# x = Block(filters=256, kernel_size=3, repetitions=3)(x)
# x = Block(filters=512, kernel_size=3, repetitions=3)(x)
# x = Block(filters=512, kernel_size=3, repetitions=3)(x)
x = tf.keras.layers.Conv2D(16, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)


y = tf.keras.layers.Dense(num_classes,activation='softmax')(x)
y2 = tf.keras.layers.Dense(pseudo_lab_class, activation='softmax')(x)

x2 = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
y_pseudo_lab = pretrainmodel(x2)
#y_pseudo_lab = tf.keras.layers.Lambda(custom_layer2)(y_pseudo_lab)
y_pseudo_lab = tf.keras.layers.Lambda(custom_layer)(y_pseudo_lab)

concat = tf.keras.layers.Concatenate()([y,y2,y_pseudo_lab])

archi = Model(inputs=input, outputs=concat)
archi.compile(loss=custom_loss(0.2), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics='accuracy')
#tb_callback = tf.keras.callbacks.TensorBoard('/content/', update_freq=1)

# tf.debugging.experimental.enable_dump_debug_info(
#    # logdir="/content/tfdbg2_logdir",
#     tensor_debug_mode="FULL_HEALTH",
#     circular_buffer_size=-1,
#     dump_root='/content/')

history = archi.fit(
  train_list_ds,
  validation_data=val_ds,
  epochs=9  #
   #callbacks=[tb_callback]
)



In [ ]:
import tensorflow_addons as tfa
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K


METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Accuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tfa.metrics.F1Score(name='f1', num_classes=2)
]



pseudo_lab_class = 1001

input = Input(shape=(224, 224,3), name="base_input")
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
x = tf.keras.layers.Conv2D(16, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

y = tf.keras.layers.Dense(1,activation='softmax')(x)

archi = Model(inputs=input, outputs=y)
archi.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=METRICS)

history = archi.fit(
  train_list_ds,
  validation_data=val_ds,
  epochs=15  #
   #callbacks=[tb_callback]
)



In [ ]:
plot_model(archi, show_shapes=True, show_layer_names=True)

In [ ]:
%load_ext tensorboard

%tensorboard --logdir /content/train

In [ ]:
tensorboard --logdir /content/

In [ ]:

def custompreprocess(features,param):
    # Resize and normalize
    print(param)
    image = tf.image.resize(in_image, (224, 224))
    return tf.cast(image, tf.float32) / 255., param


In [ ]:
vgg = NotAuxiliaryModel(num_classes=2, pseudo_lables=1280, pretrained_model=pretrainmodel)


# output_layer_1 = [layer for layer in vgg.layers if layer.name == 'output_1'][0]
# losses = {'output_1': 'categorical_crossentropy', 'output_2': custom_loss(vgg, output_layer_1.output)}

vgg.compile(loss=custom_loss(0.5), optimizer='adam',metrics=['accuracy'])
#vgg.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Compile with losses and metrics
#vgg.compile(optimizer='adam', loss=wrap_custom_loss, metrics=['accuracy'])

# Define preprocessing function

# Apply transformations to dataset
#train_list_ds = train_list_ds.map(custompreprocess).batch(32)

# Train the custom VGG model
# y_lables = np.concatenate([y for x, y in train_list_ds], axis=0)
# x_train = np.concatenate([x for x, y in train_list_ds], axis=0)

vgg.fit(
  train_list_ds,
  validation_data=val_ds,
  epochs=3
)

In [ ]:
#layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
for image_batch, labels_batch in train_list_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
import tensorflow as tf 
import numpy as np
import glob

# list_foreground = []
# list_test = []
# list_val = []
# list_background = []

# buff = glob.glob(f"./buffalo/*.jpg")
# elephant = glob.glob(f"./elephant/*.jpg")
# rhino = glob.glob(f"./rhino/*.jpg")
# zebra = glob.glob(f"./zebra/*.jpg")

# max_train = 200
# max_test = 100
# max_validation = 75

# for file in buff:  
#   list_foreground.append(file)

# for file in elephant:
#   list_background.append(file)

# for file in rhino:
#   list_background.append(file)

# for file in zebra:
#   list_background.append(file)


# labels = np.ones(len(list_foreground))
# print('lables shape',labels.shape)
# labels_background = np.zeros(len(list_background))
# print('background shape',labels_background.shape)
# labs = np.concatenate((labels, labels_background), axis=None)
# print('shape:' , labs.shape)

# processed = []
# for x in list_foreground:
#   processed.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(x)))

# for x in list_background:
#   processed.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(x)))

# filenames = tf.constant(processed)
# dataset = tf.data.Dataset.from_tensor_slices((filenames, labs))

# ds = dataset.cache()

# # Add some indices.
# ds = ds.enumerate()

# # Do a rougly 70-30 split.
# train_list_ds = ds.filter(lambda i, data: i % 10 <= 7)

# test_valid_ds = ds.filter(lambda i, data: i % 10 > 7 and i%10 <=9)

# test_list_ds = ds.filter(lambda i, data: i % 10 > 9)


# # Drop indices.
# train_list_ds = train_list_ds.map(lambda i, data: data)

# test_valid_ds = test_valid_ds.map(lambda i, data: data)

# test_list_ds = test_list_ds.map(lambda i, data: data)

# print(len(list(test_list_ds.as_numpy_iterator())))
# print(len(list(train_list_ds.as_numpy_iterator())))
# print(len(list(test_valid_ds.as_numpy_iterator())))

#list(test_list_ds.as_numpy_iterator())
